# Batch pre-computed recommendations

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys

import redis
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from tqdm.auto import tqdm

load_dotenv()

sys.path.insert(0, "..")

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    run_name: str = "003-alias-champion"
    notebook_persist_dp: str = None
    random_seed: int = 41

    redis_host: str = "localhost"
    redis_port: int = 6379
    redis_key_prefix: str = "output:i2i:"

    batch_recs_fp: str = "data/003-alias-champion/batch_recs.jsonl"

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)
        self.batch_recs_fp = f"{self.notebook_persist_dp}/batch_recs.jsonl"

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

{
  "testing": false,
  "run_name": "003-alias-champion",
  "notebook_persist_dp": "/Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/003-alias-champion",
  "random_seed": 41,
  "redis_host": "localhost",
  "redis_port": 6379,
  "redis_key_prefix": "output:i2i:",
  "batch_recs_fp": "/Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/003-alias-champion/batch_recs.jsonl"
}


# Load batch recs into Redis

In [4]:
r = redis.Redis(host=args.redis_host, port=args.redis_port, db=0, decode_responses=True)
assert (
    r.ping()
), f"Redis at {args.redis_host}:{args.port} is not running, please make sure you have started the Redis docker service"

In [5]:
def store_recommendations(file_path: str):
    with open(file_path, "r") as file:
        for line in tqdm(file):
            rec_data = json.loads(line)
            target_item = rec_data["target_item"]
            key = args.redis_key_prefix + target_item
            r.set(
                target_item,
                json.dumps(
                    {
                        "rec_item_ids": rec_data["rec_item_ids"],
                        "rec_scores": rec_data["rec_scores"],
                    }
                ),
            )


def get_recommendations(target_item):
    key = args.redis_key_prefix + target_item
    rec_data = r.get(target_item)
    if rec_data:
        return json.loads(rec_data)
    return None


def get_example_keys(count=5):
    keys = r.scan_iter(match="*", count=count)
    output = []
    for i, key in enumerate(keys, 1):
        output.append(key)
        if i >= count:
            return output

In [6]:
logger.info(f"Loading batch recs output from {args.batch_recs_fp}...")
store_recommendations(args.batch_recs_fp)

2024-10-04 15:44:26.810 | INFO     | __main__:<module>:1 - Loading batch recs output from /Users/dvq/frostmourne/fsds/fsds-recsys/chapters/l7/notebooks/data/003-alias-champion/batch_recs.jsonl...


0it [00:00, ?it/s]

In [7]:
get_recommendations(get_example_keys()[0])

{'rec_item_ids': ['B003YI458I',
  'B01953YZ0S',
  'B00A1AQJAO',
  'B00B5QGJ5S',
  'B0BMGHMP23',
  'B01GU89H6I',
  'B001ELJPPA',
  'B002GCFI2K',
  'B0041RRSP8',
  'B00CF7KVMI',
  'B002I0J8RQ',
  'B074JFY4JF',
  'B073ZNZ2JD',
  'B00ZJRHSRW',
  'B00K00NCXW',
  'B0095P5K8I',
  'B004I8EEMU',
  'B003O0KICS',
  'B00QLWZWMW',
  'B01MZ9QJ0Y',
  'B08Y5DN2FX',
  'B0B8RG61GK',
  'B000CQNWH0',
  'B07JVQLSLY',
  'B08MBPJNRF',
  'B01NCX8UUG',
  'B000B6MLTG',
  'B002FQJUQM',
  'B0038MVFYC',
  'B004HD55VA',
  'B01GOK2GQU',
  'B07DK49CCN',
  'B004OCK9KG',
  'B01MA5XZSG',
  'B08TKH1F8B',
  'B08XD54VJY',
  'B001EYUNV2',
  'B009AOFNU4',
  'B07DGSD6FZ',
  'B0050SYXSW',
  'B011I4APRE',
  'B000OCXK6A',
  'B007VYW3US',
  'B06XX6Y26M',
  'B002BSC5CU',
  'B00FATRKWM',
  'B0052WG9II',
  'B002BG1PXM',
  'B002JTX68I',
  'B01AC3ZAGI',
  'B0098F1Y3K',
  'B073S8K9GC',
  'B000J18SR2',
  'B086N3DT9H',
  'B06XC1WNPT',
  'B008M4BNQG',
  'B06Y3RB65T',
  'B000E36ONK',
  'B01GW8ZC2O',
  'B00005M8TL',
  'B07MFMQZ7M',
  'B00TI